# Commands

In [1]:
# cd twinews-logs ; jupython -o nohup-tfidf-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/tfidf.ipynb

# Init

In [2]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
isNotebook = '__file__' not in locals()

In [4]:
TEST = isNotebook # isNotebook, True, False

In [5]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from twinews.utils import *
from twinews.models.ranking import *

In [6]:
from nlptools.topicmodeling import *
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
logger = Logger(tmpDir('logs') + "/tfidf.log") if isNotebook else Logger("tfidf-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# Config

In [21]:
config = \
{
    'splitVersion': 2,
    
    'maxUsers': 2 if TEST else None, # Sub-sampling
    'minDF': None if TEST else None, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': None if TEST else None, # Remove top 300 voc elements
    
    'lowercase': False if TEST else True,
    'doLemmatization': False if TEST else False,
    'sublinearTF': True,
    
    'dimensions': None, # 20, 100, 500, None (means no dimensionality reduction)
    
    'distance': 'cosine', # 'cosine', 'euclidean', 'kl', 'js'
    'historyRef': 30, # 1, 1.0, 0.6, 0.3, 3, 10, 30
}

In [9]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

# Getting data

In [10]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 32.09s | message: Eval data loaded
--> toc total duration: 32.37s | message: Got Twinews evaluation data
{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


In [13]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [14]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

28 urls for trainNewsList
1989 urls for testNewsList
2017 urls for newsList


In [15]:
# We get sentences:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger)

twinews news (version 1.0) initialised.
  0% [                    ]
  9% [=                   ] (36.5s left)
 19% [===                 ] (30.934s left)
 29% [=====               ] (27.201s left)
 39% [=======             ] (23.218s left)
 49% [=========           ] (19.353s left)
 59% [===========         ] (15.5s left)
 69% [=============       ] (11.692s left)
 79% [===============     ] (7.77s left)
 89% [=================   ] (3.97s left)
 99% [=================== ] (0.134s left)
100% [====================] (total duration: 38.6s, mean duration: 0.019s)
[
  [ [ West, Ham, ..., decline, . ], [ The, club, ..., staff, . ], ..., [ There, was, ..., Monday, . ], [ Bilic, fate, ..., Monday, . ] ],
  [ [ What, missing, ..., room, . ], [ ", I, ..., late, ! ], ..., [ I, know, ..., hurting, . ], [ I, know, ..., of, . ] ],
  ...,
  [ [ In, __int_4__, ..., project, . ], [ They, gathered, ..., years, . ], ..., [ To, order, ..., call, __int_3__ ], [ Free, UK, ..., only, . ] ],
  [ [ In, Rural, ..

In [16]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ West, Ham, ..., Monday, . ], [ What, missing, ..., of, . ], ..., [ In, __int_4__, ..., only, . ], [ In, Rural, ..., state, . ] ]


In [17]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

[ [ West, Ham, ..., Monday, . ], [ What, missing, ..., of, . ], ..., [ In, __int_4__, ..., only, . ], [ In, Rural, ..., state, . ] ]


In [18]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

[ [ West, Ham, ..., Monday, . ], [ What, missing, ..., of, . ], ..., [ In, __int_4__, ..., only, . ], [ In, Rural, ..., state, . ] ]


In [19]:
# Filtering the corpus:
if config['minDF'] is not None or config['maxDF'] is not None:
    docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                        removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
    for doc in docs: assert len(doc) > 0
    bp(docs, logger)

In [20]:
tt.tic("Data preprocessed")

--> tic: 5m 38.019s | message: Data preprocessed


338.02

# Infering vectors

In [22]:
vectorizer = TfidfVectorizer\
(
    sublinear_tf=config['sublinearTF'],
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    # lowercase=True, # Doesn't work because we erased preprocessor
)
vectors = vectorizer.fit_transform(docs)
assert vectors.shape[0] == len(newsList)

In [23]:
tt.tic("TFIDF vectors generated")

--> tic: 4m 19.73s | message: TFIDF vectors generated


259.73

# Dimensionality reduction

# Making a dict url --> topic vector

In [ ]:
assert len(urlsToInfere) == len(inferedVectors)
urlsVectors = dict()
for i in range(len(urlsToInfere)):
    urlsVectors[urlsToInfere[i]] = inferedVectors[i]

# Ranking

In [ ]:
if False and isNotebook:
    config['historyRef'] = 1.0

In [ ]:
historyRef = config['historyRef']
assert (isinstance(historyRef, int) and historyRef >= 1) or (isinstance(historyRef, float) and historyRef > 0.0 and historyRef <= 1.0)

In [ ]:
# Read the doc!
rankings = usersRankingsByHistoryDistance\
(
    trainUsers,
    candidates,
    historyRef,
    urlsVectors,
    distanceMetric=config['distance'],
    logger=logger,
)

In [ ]:
bp(rankings, logger, 4)

In [ ]:
tt.tic("Rankings done")

# Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())